In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule, ColorScaleRule
from openpyxl.utils import get_column_letter
import seaborn as sns
sns.set()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


TARGET_F = "renewal" #Whether an account/member renews (binary outcome)
TARGET_F_DESCRIPTION = "Renewal"
TARGET_A = "trunc_months_between_unlocks" #When the renewal occurs in months from first unlock (timing outcome)
TARGET_A_DESCRIPTION = "Number of Months from First Unlock"


DATA = "/Users/patbradley/Documents/Masters/Northwestern/MSDS 498/Project/renewal_data_deidentified.csv"
DATA_DICTIONARY = "/Users/patbradley/Documents/Masters/Northwestern/MSDS 498/Project/renewal_data_dictionary.csv"

df = pd.read_csv(DATA)
df_dict = pd.read_csv(DATA_DICTIONARY, encoding="ISO-8859-1")

var_to_desc = dict(zip(df_dict["Variable"], df_dict["Description"]))


# print(df.head().T)

In [131]:
dt = df.dtypes
# print( dt )

objList = []
intList = []
floatList = []

for i in dt.index :
    if i in ( [ TARGET_F,TARGET_A] ) : continue
    if dt[i] in (["object"]) : objList.append( i )
    if dt[i] in (["float64"]) : floatList.append( i )
    if dt[i] in (["int64"]) : intList.append( i )

In [135]:
##Object Exploration
#Before Data Impute
i = "sex"

# print(" Class = ", i )
g = df.groupby( i )
#print( g[i].count() )
x = g[ TARGET_F ].mean()
# print(f"{TARGET_F_DESCRIPTION} Prob", x )
# print( " ................. ")
x = g[ TARGET_A ].mean()
# print( f"{TARGET_A_DESCRIPTION}", x )
# print(" ===============\n\n\n ")


In [139]:
for i in intList :
   # print("Class = ", i ) 
   desc = var_to_desc.get(i, "No description found")
   # print("Description:", desc,"\n")
   g = df.groupby( i )
   #print( g[i].count() )
   x = g[ TARGET_F ].mean()
   # print(f"{TARGET_F_DESCRIPTION} Prob", x )
   # print( " ................. ")
   x = g[ TARGET_A ].mean()
   # print( f"{TARGET_A_DESCRIPTION}", x )
   # print(" ===============\n\n\n ")

In [119]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return s
    return (s.replace("Õ", "'")
             .replace("’", "'")
             .replace("“", '"')
             .replace("”", '"'))

results_frames = []

for i in intList:
    desc = normalize_text(var_to_desc.get(i, "No description found"))

    agg = (
        df.groupby(i)[[TARGET_F, TARGET_A]]
          .mean()
          .reset_index()
          .rename(columns={
              i: "GroupValue",
              TARGET_F: f"{TARGET_F_DESCRIPTION} Prob",
              TARGET_A: f"{TARGET_A_DESCRIPTION}",
          })
    )

    # Add Class andDescription columns
    agg.insert(0, "Class", i)
    agg.insert(1, "Description", desc)

    results_frames.append(agg)

    # Blank row for sep
    blank = pd.DataFrame([{col: "" for col in agg.columns}])
    results_frames.append(blank)

out_df = pd.concat(results_frames, ignore_index=True)

num_cols = [f"{TARGET_F_DESCRIPTION} Prob", f"{TARGET_A_DESCRIPTION}"]
for c in num_cols:
    out_df[c] = pd.to_numeric(out_df[c], errors="coerce")

out_df[num_cols] = out_df[num_cols].round(6)

# Save to Excel in the working directory
out_path = "grouped_summary.xlsx"
out_df.to_excel(out_path, index=False)

# Color the fully blank separator rows red
wb = load_workbook(out_path)
ws = wb.active
red = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    if all(cell.value in (None, "") for cell in row):
        for cell in row:
            cell.fill = red

wb.save(out_path)
print(f"✅ Results saved to {out_path}")


✅ Results saved to grouped_summary.xlsx


In [121]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.utils import get_column_letter


out_path = "grouped_summary.xlsx"

RENEWAL_COL_HEADER = "Renewal Prob"                 
MONTHS_COL_HEADER  = "Number of Months from First Unlock"


wb = load_workbook(out_path)
ws = wb.active

header_map = {cell.value: idx for idx, cell in enumerate(ws[1], start=1) if cell.value}

def col_idx_for(header):
    if header not in header_map:
        raise ValueError(f"Couldn't find column '{header}' in header row.")
    return header_map[header]

renewal_idx = col_idx_for(RENEWAL_COL_HEADER)
months_idx  = col_idx_for(MONTHS_COL_HEADER)
renewal_col = get_column_letter(renewal_idx)
months_col  = get_column_letter(months_idx)

# Find class blocks separated by fully blank rows
blocks = []
start = 2
for r in range(2, ws.max_row + 1):
    row_blank = all(ws.cell(r, c).value in (None, "") for c in range(1, ws.max_column + 1))
    if row_blank:
        if r - 1 >= start:
            blocks.append((start, r - 1))
        start = r + 1
if start <= ws.max_row:  # tail block
    blocks.append((start, ws.max_row))

# Define color rules
renewal_rule = lambda rng: ColorScaleRule(
    start_type="min", start_color="F8696B",    # red (low)
    mid_type="percentile", mid_value=50, mid_color="FFEB84",  # yellow
    end_type="max", end_color="63BE7B"         # green (high)
)
months_rule = lambda rng: ColorScaleRule(
    start_type="min", start_color="63BE7B",    # green (low)
    mid_type="percentile", mid_value=50, mid_color="FFEB84",  # yellow
    end_type="max", end_color="F8696B"         # red (high)
)

# Apply conditional formatting per block
for (r0, r1) in blocks:
    if r1 < r0:
        continue
    rng_renewal = f"{renewal_col}{r0}:{renewal_col}{r1}"
    rng_months  = f"{months_col}{r0}:{months_col}{r1}"
    ws.conditional_formatting.add(rng_renewal, renewal_rule(rng_renewal))
    ws.conditional_formatting.add(rng_months,  months_rule(rng_months))

# Apply grey fill to blank separator rows
grey = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    if all(cell.value in (None, "") for cell in row):
        for cell in row:
            cell.fill = grey

# Auto adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        try:
            if cell.value:
                length = len(str(cell.value))
                if length > max_length:
                    max_length = length
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2


wb.save(out_path)
print(f"✅ Applied per-class formatting, grey separators, and auto-fit columns → {out_path}")


✅ Applied per-class formatting, grey separators, and auto-fit columns → grouped_summary.xlsx
